In [1]:
import os
from dotenv import load_dotenv

In [2]:
try:
    load_dotenv()
    print("key initialized")
except Exception as e:
    print(f"key failed to initialize")

key initialized


Why do Agents need Tools?
The Problem

Without tools, the agent's knowledge is frozen in time — it can't access today's news or your company's inventory. It has no connection to the outside world, so the agent can't take actions for you.

The Solution: Tools are what transform your isolated LLM into a capable agent that can actually help you get things done.

In this notebook, you'll:

✅ Turn your Python functions into Agent tools
✅ Build an Agent and use it as a tool in another agent
✅ Build your first multi-tool agent
✅ Explore the different tool types in ADK

In [3]:
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Key Imported")
except Exception as e:
    print(f"Error connecting to key {e}")

Key Imported


In [4]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("Import successful")

Import successful


In [5]:
def show_python_code_and_result(response):
    """
    Extract and display Python code and execution results from agent responses.
    
    This function iterates through API response objects and looks for function call results
    that contain executed Python code and its output.
    
    Args:
        response: List of response objects from the agent/API
    """
    # Loop through each response item in the response list
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        # This validates that:
        # 1. response[i] has content.parts (contains response parts)
        # 2. response[i].content.parts[0] exists (first part exists)
        # 3. First part has function_response (function was called)
        # 4. function_response has a response object (execution result exists)
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            # Extract the response code/result from the function response
            response_code = response[i].content.parts[0].function_response.response
            
            # Check if response has a "result" key and it's not empty
            if "result" in response_code and response_code["result"] != "```":
                # Check if this is Python code (contains "tool_code" key)
                if "tool_code" in response_code["result"]:
                    # Print the generated Python code (remove "tool_code" label)
                    print(
                        "Generated Python Code >>",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    # Print the response if it's not code (e.g., plain text result)
                    print("Generated python response >>", response_code["result"])

print("helper function defined")

helper function defined


In [6]:
retry_config = types.HttpRetryOptions(
    attempts = 5, # Max retry attempts
    exp_base = 7, # delay multiplier
    initial_delay = 1, # initial delay before first retry in seconds
    http_status_codes = [429, 500, 503, 504] # retry on these HTTP errors
)

 Section 2: What are Custom Tools?¶
Custom Tools are tools you build yourself using your own code and business logic. Unlike built-in tools that come ready-made with ADK, custom tools give you complete control over functionality.

When to use Custom Tools?

Built-in tools like Google Search are powerful, but every business has unique requirements that generic tools can't handle. Custom tools let you implement your specific business logic, connect to your systems, and solve domain-specific problems. ADK provides multiple custom tool types to handle these scenarios.

# 2.1: Building Custom Function Tools
Example: Currency Converter Agent
This agent can convert currency from one denomination to another and calculates the fees to do the conversion. The agent has two custom tools and follows the workflow:

1. Fee Lookup Tool - Finds transaction fees for the conversion (mock)
2. Exchange Rate Tool - Gets currency conversion rates (mock)
3. Calculation Step - Calculates the total conversion cost including the fees

<img src="/Users/mario/Desktop/kaggle/Day-2-A/1.png"/>

# defining a tool

any python function can become an agent tool, we have to follow basic guidelines

1. Create a Python function.
2. Follow the best practices listed below.
3. add function to agent's tools=[] list and ADK handles the rest automatically.

# ADK best practices

1. Dictionary Return: Tools return {"status": "success", "data": ...} or {"status": "error", "error_message": ..}
2. Clear Docstrings: LLMs use docstrings when and how to use tools.
3. Type Hints: Enable ADK to generate proper scheme (str, dict, etc).
4. Error Handling: Structured error responses help LLMs handle failures gracefully. 

# testing on our tool

In [7]:
def get_fee_for_payment_method(method:str) -> dict:
    """ 
        Looks up the transaction fee percentage for a given payment method.

        this tool simulates looking up a company's internal fee structure based on the name of the payment method
        provided by the user.

        Args:
            method: The name of the payment method. It should be descriptive. e.g., "Platinum credit card" or "bank transfer".
        Returns:
            Dictionary with the status and fee information.
            Success: {"status": "success", "fee_percentage": 0.02}
            Error: {"status": "error", "error_message": "Payment method not found"} 
    """

    # this simulates a company's internal fee structure.

    fee_database = {
        "platinum credit card": 0.02, # 2%
        "gold debit card": 0.035, # 3.5%
        "bank transfer": 0.01 # 1%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "Error",
            "error_message": f"Payment method '{method}' not found",
        }
print("fee lookup function created")
print(f"test: {get_fee_for_payment_method('platinum credit card')}")






fee lookup function created
test: {'status': 'success', 'fee_percentage': 0.02}


In [8]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    # Static data simulating a live exchange rate API
    # In production, this would call something like: requests.get("api.exchangerates.com")
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    # Input validation and processing
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }


print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 0.93}


Now let's create our currency agent. Pay attention to how the agent's instructions reference the tools:

Key Points:

The tools=[] list tells the agent which functions it can use
Instructions reference tools by their exact function names (e.g., get_fee_for_payment_method())
The agent uses these names to decide when and how to call each tool

In [9]:
# currency agent
currency_agent = LlmAgent(
    name="CurrencyAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=""" You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly. """,
    tools=[get_fee_for_payment_method, get_exchange_rate]
)
print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [10]:
runner = InMemoryRunner(agent=currency_agent)
_ = await runner.run_debug("I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?")


 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?


CurrencyAgent > I will receive 455.70 Euros.

Here's the breakdown:

1.  **Fee Calculation:** The transaction fee for using a Platinum Credit Card is 2.00%. This amounts to 10.00 USD (500 USD * 0.02).
2.  **Amount After Fee:** After deducting the fee, you will have 490.00 USD remaining (500 USD - 10.00 USD).
3.  **Currency Conversion:** Using the exchange rate of 1 USD to 0.93 EUR, the final amount is 455.70 EUR (490.00 USD * 0.93).


# Excellent! Our agent now uses custom business logic with structured responses.

# Section 3 - Improving agent Reliability with code

The agent's instruction says "calculate the final amount after fees" but LLMs aren't always reliable at math. They might make calculation errors or use inconsistent formulas.

💡 Solution: Let's ask our agent to generate a Python code to do the math, and run it to give us the final result! Code execution is much more reliable than having the LLM try to do math in its head!

<img src="/Users/mario/Desktop/kaggle/Day-2-A/2.png"/>

In [12]:
# ADK has a built-in Code Executor capable of running code in a sandbox. Note: This uses Gemini's
# Code Execution capability.

# Let's create a calculation_agent which takes in a Python code and uses the BuiltInCodeExecutor to run it.

In [13]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options= retry_config
    ),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor() # use teh built in code executor tool, this gives the agent code execution ability
)

# section 3.2 - Updating the Agent's instruction and toolset 

We'll do two key actions:

1) Update the currency_agent's instructions to generate Python code<br>
a) Original: "Calculate the final amount after fees" (vague math instructions)<br>
b) Enhanced: "Generate a Python code to calculate the final amount .. and use the calculation_agent to run the code and compute final amount"

2) Add the calculation_agent to the toolset

ADK lets you use any agent as a tool using AgentTool.

a) Add AgentTool(agent=calculation_agent) to the tools list <br>
b) The specialist agent appears as a callable tool to the root agent



In [14]:
enhanced_currency_agent = LlmAgent(
    name="EnhancedCurrencyAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[get_fee_for_payment_method, get_exchange_rate, AgentTool(agent=calculation_agent),],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [15]:
# enhanced runner

enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)


In [18]:
try:
    response = await enhanced_runner.run_debug("Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.")
except Exception as e:
    print(f"print the error {e}")


 ### Continue session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.


EnhancedCurrencyAgent > The final amount is 103,430.25 INR.

Here's the breakdown of the calculation:
*   **Transaction Fee**: A 1% fee was applied to the transaction, which amounts to 12.50 USD.
*   **Amount after Fee**: After deducting the fee, the amount to be transferred is 1237.50 USD.
*   **Exchange Rate**: The exchange rate used for the conversion is 1 USD = 83.58 INR.
*   **Final Converted Amount**: The final amount in INR is 103,430.25 INR.


Excellent! Notice what happened:

When the Currency agent calls the CalculationAgent, it passes in the generated Python code
The CalculationAgent in turn used the BuiltInCodeExecutor to run the code and gave us precise calculations instead of LLM guesswork!
Now you can inspect the parts of the response that either generated Python code or that contain the Python code results, using the helper function that was defined near the beginning of this notebook:

In [19]:
show_python_code_and_result(response)

Generated python response >> USD Amount: 1250
Fee Percentage: 1.0%
Fee Amount in USD: 12.50
Amount to be transferred in USD: 1237.50
Exchange Rate: 1 USD = 83.58 INR
Final Amount in INR: 103430.25


# 3.3: Agent Tools vs Sub-Agents: What's the Difference?
This is a common question! Both involve using multiple agents, but they work very differently:

1) Agent Tools (what we're using): <br>

Agent A calls Agent B as a tool<br>
Agent B's response goes back to Agent A<br>
Agent A stays in control and continues the conversation<br>
Use case: Delegation for specific tasks (like calculations)<br>

2) Sub-Agents (different pattern):<br>

Agent A transfers control completely to Agent B<br>
Agent B takes over and handles all future user input<br>
Agent A is out of the loop<br>
Use case: Handoff to specialists (like customer support tiers)<br>
In our currency example: We want the currency agent to get calculation results and continue working with them, so we use Agent Tools, not sub-agents.

# section 4; Complete Guide to ADK tool types
Now that you've seen tools in action, let's understand the complete ADK toolkit: <br>

It's broadly divided into two categories: Custom tools and Built-in tools

<li>1. Custom tool</li>
<img src="/Users/mario/Desktop/kaggle/Day-2-A/3.png"/>

**What**: Tools you build yourself for specific needs

**Advantage**: Complete control over functionality — you build exactly what your agent needs

#### **Function Tools** ✅ (You've used these!)
- **What**: Python functions converted to agent tools
- **Examples**: `get_fee_for_payment_method`, `get_exchange_rate`
- **Advantage**: Turn any Python function into an agent tool instantly

#### **Long Running Function Tools**
- **What**: Functions for operations that take significant time
- **Examples**: Human-in-the-loop approvals, file processing
- **Advantage**: Agents can start tasks and continue with other work while waiting

#### **Agent Tools** ✅ (You've used these!)
- **What**: Other agents used as tools
- **Examples**: `AgentTool(agent=calculation_agent)`
- **Advantage**: Build specialist agents and reuse them across different systems

#### **MCP Tools**
- **What**: Tools from Model Context Protocol servers
- **Examples**: Filesystem access, Google Maps, databases
- **Advantage**: Connect to any MCP-compatible service without custom integration

#### **OpenAPI Tools**
- **What**: Tools automatically generated from API specifications
- **Examples**: REST API endpoints become callable tools
- **Advantage**: No manual coding — just provide an API spec and get working tools

<li>2. Built-in tools</li>
<img src="/Users/mario/Desktop/kaggle/Day-2-A/4.png"/>

**What**: Pre-built tools provided by ADK

**Advantage**: No development time — use immediately with zero setup

#### **Gemini Tools** ✅ (You've used these!)
- **What**: Tools that leverage Gemini's capabilities
- **Examples**: `google_search`, `BuiltInCodeExecutor`
- **Advantage**: Reliable, tested tools that work out of the box

#### **Google Cloud Tools** [needs Google Cloud access]
- **What**: Tools for Google Cloud services and enterprise integration
- **Examples**: `BigQueryToolset`, `SpannerToolset`, `APIHubToolset`
- **Advantage**: Enterprise-grade database and API access with built-in security

#### **Third-party Tools**
- **What**: Wrappers for existing tool ecosystems
- **Examples**: Hugging Face, Firecrawl, GitHub Tools
- **Advantage**: Reuse existing tool investments — no need to rebuild what already exists
